In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from vulcan.src.timeseries import dtw_analysis, plot_rolling_moments
from vulcan.src.visual import plot_distribution
from vulcan.src.data import summary_df
from vulcan.src.ml import train_xgb_rf


%load_ext autoreload
%autoreload 2
%matplotlib inline
%xmode plain

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Exception reporting mode: Plain


In [7]:
df = pd.read_csv('/users/mzoffoli/Desktop/roll_adj_prices.csv', index_col=0)
df.index = pd.to_datetime(df.index)
df = df.ffill()

display(summary_df(df))

,dtype,n_missing,pct_missing,n_inf,pct_inf,n_unique,memory_usage,mean,std,min,25%,50%,75%,max
CMEF:B001#9,float64,1044,34.75%,0,0.0%,1762,48064,0.66,0.24,0.13,0.53,0.67,0.78,1.39
CMEF:CSC01#9,float64,1044,34.75%,0,0.0%,1729,48064,2.63,0.54,1.68,2.16,2.60,2.97,4.65
CMEF:CU01#9,float64,1044,34.75%,0,0.0%,1487,48064,1.04,0.62,0.23,0.40,0.84,1.66,1.96
CMEF:DC01#9,float64,1044,34.75%,0,0.0%,1738,48064,22.46,3.59,14.76,19.71,22.49,24.39,35.75
CMEF:AD001#9,float64,1044,34.75%,0,0.0%,1774,48064,0.56,0.27,0.10,0.28,0.61,0.81,1.21
EURF:DET01#9,float64,864,28.76%,0,0.0%,2063,48064,1142.47,920.58,59.80,160.43,988.00,1974.91,3866.50
ZAXF:EB01#9,float64,1,0.03%,0,0.0%,2568,48064,5481.18,966.95,4104.97,4690.64,5175.79,6286.05,8696.75
EURF:FAC01#9,float64,1,0.03%,0,0.0%,2870,48064,20335.81,16891.47,22.26,5205.67,22740.80,32219.90,102695.20
CMEF:HRC01#9,float64,1044,34.75%,0,0.0%,1515,48064,677.93,299.42,243.48,401.68,725.49,925.66,1442.03
EURF:ITP01#9,float64,81,2.7%,0,0.0%,2785,48064,385.47,215.64,82.60,151.10,430.46,540.82,1637.84


In [8]:
best_model, perf = train_xgb_rf(df=df.tail(200), target="CMEF:CU01#9")

[I 2025-07-14 15:50:46,983] A new study created in memory with name: no-name-6fc9e8dd-bcdf-4d5a-a845-52b19146958b
[I 2025-07-14 15:50:47,918] Trial 0 finished with value: 0.03470262270672385 and parameters: {'n_estimators': 679, 'learning_rate': 0.03857702875131217, 'subsample': 0.8522727191231231, 'colsample_bynode': 0.7266244497198018, 'max_depth': 6, 'min_child_weight': 8.66594130997461, 'reg_lambda': 7.849206433507661, 'gamma': 0.08600085583898465}. Best is trial 0 with value: 0.03470262270672385.
[I 2025-07-14 15:50:49,372] Trial 1 finished with value: 0.03471213958167298 and parameters: {'n_estimators': 1141, 'learning_rate': 0.010016695042949773, 'subsample': 0.9222882746040818, 'colsample_bynode': 0.8451142263839891, 'max_depth': 6, 'min_child_weight': 3.8455326723979883, 'reg_lambda': 3.672633291903572, 'gamma': 1.9416924393155055}. Best is trial 0 with value: 0.03470262270672385.
[I 2025-07-14 15:50:49,709] Trial 2 finished with value: 0.034707977190646166 and parameters: {'n

In [9]:
display(perf)

,fold,mae,rmse
0,0,0.051228,0.054221
1,1,0.010019,0.011699
2,2,0.030209,0.032957
3,3,0.046190,0.049475
4,test,0.135773,0.155798


In [10]:
best_model

,learning_rate,0.19827295693892708
,subsample,0.5443261177749809
,colsample_bynode,0.616653852608256
,reg_lambda,0.30172114244294246
,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bytree,None
,device,None
